<a href="https://colab.research.google.com/github/marcosdiezw/Investment_optimization_problem/blob/main/Optimization_problem_solving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
#  **<center> <font color= purple size=10> Bootcamp de otimização</font>**
***

<center>
<img src="https://www.enacom.com.br/wp-content/uploads/2021/01/Logo_Enacom.svg" align="middle" width="950">
</center>

# Enunciado

Considerando um capital para investimento de R$1.000.000 e as seguintes opções de investimento:

| **Opção** | **Descrição** | **Custo do investimento (R$)**| **Retorno esperado (R\$)** |
|:----------:|:------------|:-------------------:|:-------------------------:|
| 1 | Ampliação da capacidade do armazém ZDP em 5% | 470.000| 410.000 |
| 2 | Ampliação da capacidade do armazém MGL em 7% | 400.000| 330.000 |
| 3 | Compra de empilhadeira | 170.000| 140.000 |
| 4 | Projeto de P&D I | 270.000| 250.000 |
| 5 | Projeto de P&D II | 340.000| 320.000 |
| 6 | Aquisição de novos equipamentos | 230.000| 320.000 |
| 7 | Capacitação de funcionários | 50.000| 90.000 |
| 8 | Ampliaçaão da estrutura da carga rodoviária | 440.000| 190.000 |


<br>

Desenvolva um algoritmo de otimização para seleccionar os projetos que maximizam o retorno total esperado, considerando que:
* Se o investimento 1 for selecionado, então o investimento 5 não deve ser;
* Se o investimento 2 for selecionado, então o investimento 4 também deve ser.

# Resolução

*O arquivo de Power Point para a apresentação se encontra no seguinte link:*

https://drive.google.com/file/d/1jkye40O64aLHrkLRzlQad-eZfUtMy5Kb/view

***

O seguinte algoritmo foi criado para detectar o pacote de investimentos ótimos (ou os pacotes, se forem vários) após receber *quaisquer conjuntos de dados* con os três elementos a seguir: <br>

1) Lista do total de investimentos possíveis, com seus números de identificação, descrição, custos e retornos esperados.

2) Gasto total máximo (orçamento).

3) Restrições.


In [19]:
import pandas as pd
import numpy as np
from itertools import combinations
import unittest

## Dados de entrada do nosso problema em particular:

1) Conjunto de investimentos possíveis:

In [20]:
proposals = [
    [1,'Armazem_ZDP', 470000, 410000],
    [2,'Armazem_MGL',400000,330000],
    [3,'Compra_empilhadeira',170000,140000],
    [4,'P&D_I',270000,250000],
    [5,'P&D_II',340000,320000],
    [6,'Novos_equipamentos',230000,320000],
    [7,'Capacitacao_funcionarios',50000,90000],
    [8,'Ampliacao_estr_carga_rod',440000,190000]
    ]


2) Gasto total máximo

In [21]:
max_expenses = 1000000

3) Restrições

Métodos auxiliares:

In [22]:
  def filter_by_incompatible_elements(unfiltered_combinations, element1, element2):
    '''Recebe conjunto de combinações e o par de elementos incompatíves.
    Retorna combinações já filtradas'''
    warehouse1=[]
    for arrangement in unfiltered_combinations:
      if (element1 in arrangement) and (element2 in arrangement):
        continue
      else:
        warehouse1.append(arrangement)
    return(warehouse1)  
  
  def filter_by_dependents_elements(unfiltered_combinations,independent_element,dependent_element,):
    warehouse2=[]
    '''Recebe conjunto de combinações, um elemento possível e outro que necesariamente deve
    estar na combinação se está presente o primeiro. Retorna combinações já filtradas'''
    for arrangement in unfiltered_combinations:
      if (independent_element in arrangement) and (dependent_element not in arrangement):
        continue
      else:
        warehouse2.append(arrangement)
    return(warehouse2)
  


In [23]:
# Constraints, própriamente:

constraint1 = [filter_by_incompatible_elements, 1,5]
constraint2 = [filter_by_dependents_elements, 2, 4]

constraints_group = [constraint1, constraint2]

## Classes

In [24]:
# Classe investimento. Os objetos dessa classe são cada um dos investimentos unitários.

class Investment:
  def __init__(self,identifier_number,description,cost, gain):
    self._identifier_number = identifier_number
    self._description = description
    self._cost = cost
    self._gain = gain
  
  @property 
  def identifier_number(self) -> int:
    return self._identifier_number

  @property 
  def description(self) -> str:
    return self._description
  
  @property 
  def cost(self) -> int:
    return self._cost
  
  @property 
  def gain(self) -> int:
    return self._gain

In [25]:
# Classe pacote de investimentos. Os objetos da classe são pacotes de investimentos.

class InvestmentPackage:
  def __init__(self,combination,budget,profit):
    self._combination = combination
    self._budget = budget
    self._profit = profit

  @property
  def combination(self) -> list:
    return self._combination
  
  @property
  def budget(self) -> float:
    return self._budget
  
  @property
  def profit(self) -> float:
    return self._profit

In [26]:
# Classe Cojunto de investimentos. Os objetos são compostos por vetores do tipo:
# [conjunto_de_investimentos, gasto_máximo_permitido, restrições]

class InvestmentsSet:
  def __init__(self,investments_set,max_spending, constraints):
    self._investments_set = investments_set
    self._max_spending = max_spending
    self._constraints = constraints

  @property
  def investments_set(self):
    return self._investments_set

  @property
  def max_spending(self):
    return self._max_spending
  
  @property
  def constraints(self):
    return self._constraints


  @property
  def get_elements_combinations(self,):
    vector = np.array(self.investments_set)
    numbers_vector = vector[:,0]
    numbers_vector = numbers_vector.astype(int)
    numbers = list(numbers_vector)
    return numbers
 
  @property
  def min_number_elements(self):
    vector = np.array(self.investments_set)
    costs_vector = vector[:,2]
    costs_vector = costs_vector.astype(int)
    costs = list(costs_vector)
    costs.sort(reverse=True)
    capacity = 0
    n=0
    for t in costs:
      if capacity + t <= self.max_spending:
        capacity = capacity + t
        n+=1
      else:
        break
    return n

  @property
  def max_number_elements(self):
    vector = np.array(self.investments_set)
    costs_vector = vector[:,2]
    costs_vector = costs_vector.astype(int)
    costs = list(costs_vector)
    costs.sort(reverse=False)
    capacity = 0
    m=0
    for t in costs:
      if capacity + t <= self.max_spending:
        capacity = capacity + t
        m+=1
      else:
        break
    return m

  @property
  def combine(self):
    '''Recebe conjunto de investimentos, gasto máximo e restrições.
    Retorna todas as combinações possíveis dos números de identificação.'''
    combinations_group = []
    for u in range (self.min_number_elements, self.max_number_elements + 1):
      n_elements_combinations = combinations(self.get_elements_combinations,u)
      for combination in n_elements_combinations:
        combination_i = list(combination)
        combinations_group.append(combination_i)
    return combinations_group
  
  def enforce_constraints(self,combinations_group):
    '''Recebe conjunto de combinações. Retorna outro conjunto filtrado segundo as restrições'''
    restricted_combinations = combinations_group
    for constraint in self.constraints:
      restricted_combinations = (constraint[0])(restricted_combinations, constraint[1], constraint[2])
    return restricted_combinations
  
  @staticmethod
  def detaile_package(investment_set, combination):
    '''Recebe conjunto de investimentos unitários e combinações.
    Retorna pacotes de investimentos detalhados'''
    budget = 0
    profit = 0
    for a in combination:
      budget = budget + investment_set[a-1][2]
      profit = profit + investment_set[a-1][3]
    package = [combination, budget, profit]
    return(package)


  def detail_all_packages(self, combinations_group):
    detailed_packages = []
    for combination in combinations_group:
      detailed_investment = self.detaile_package(self.investments_set, combination)
      detailed_packages.append(detailed_investment)
    return(detailed_packages)
  
  def remove_off_budget_packages(self, detailed_packages):
    '''Recebe pacotes de investimentos detalhados. Remove pacotes fora do orçamento e
    retorno pacotes dentro do orçamento'''
    warehouse = []
    for investment_package in detailed_packages:
      if investment_package[1] > self.max_spending:
        continue
      else:
        warehouse.append(investment_package)
    return(warehouse)
  
  def optimize(self, on_budget_packages):
    '''Recebe um conjunto de pacotes de investimentos detalhados
    e retorna o ótimo'''
    optimal_profit = 0
    optimal_cost = 0
    optimal = []
    several_optimals = []
    for package in on_budget_packages:
      if package[2] < optimal_profit:
        continue
      elif package[2] > optimal_profit:
        optimal_profit = package[2]
        optimal_cost = package[1]
        optimal = package
        several_optimals = []
        several_optimals.append(package)
      elif package[2] == optimal_profit:
        if package[1] > optimal_cost:
          continue
        elif package[1] == optimal_cost:
          several_optimals.append(package)
    if len(several_optimals) > 1:
      return several_optimals
    else:
      return optimal
    
  def get_optimal(self):
    '''Recebe um conjunto de investimentos unitarios, o gasto máximo e
    e retorna a combinação de investimentos ótima'''
    filtered_packages = self.enforce_constraints(self.combine)
    detailed_packages = self.detail_all_packages(filtered_packages)
    on_budget_packages = self.remove_off_budget_packages(detailed_packages)
    return self.optimize(on_budget_packages)



## Sequência de resolução

In [27]:
# A partir das informações de entrada são extraidos os números de identificação,
# as quantidades mínima e máxima de elementos por combinação e
# as combinações possíveis.

combinaveis = InvestmentsSet(proposals,max_expenses,constraints_group)

numeros = combinaveis.get_elements_combinations
q_max = combinaveis.max_number_elements
q_min = combinaveis.min_number_elements

print('Elementos:',numeros,
      '\nQuantidade mínima de elementos por combinação:',q_min,
      '\nQuantidade máxima de elementos por combinação:',q_max)

Elementos: [1, 2, 3, 4, 5, 6, 7, 8] 
Quantidade mínima de elementos por combinação: 2 
Quantidade máxima de elementos por combinação: 4


In [28]:
combinacoes = combinaveis.combine
print(combinacoes)

[[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [4, 5], [4, 6], [4, 7], [4, 8], [5, 6], [5, 7], [5, 8], [6, 7], [6, 8], [7, 8], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6], [1, 2, 7], [1, 2, 8], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 3, 7], [1, 3, 8], [1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 4, 8], [1, 5, 6], [1, 5, 7], [1, 5, 8], [1, 6, 7], [1, 6, 8], [1, 7, 8], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 7], [2, 3, 8], [2, 4, 5], [2, 4, 6], [2, 4, 7], [2, 4, 8], [2, 5, 6], [2, 5, 7], [2, 5, 8], [2, 6, 7], [2, 6, 8], [2, 7, 8], [3, 4, 5], [3, 4, 6], [3, 4, 7], [3, 4, 8], [3, 5, 6], [3, 5, 7], [3, 5, 8], [3, 6, 7], [3, 6, 8], [3, 7, 8], [4, 5, 6], [4, 5, 7], [4, 5, 8], [4, 6, 7], [4, 6, 8], [4, 7, 8], [5, 6, 7], [5, 6, 8], [5, 7, 8], [6, 7, 8], [1, 2, 3, 4], [1, 2, 3, 5], [1, 2, 3, 6], [1, 2, 3, 7], [1, 2, 3, 8], [1, 2, 4, 5], [1, 2, 4, 6], [1, 2, 4, 7], [1, 2, 4, 8], [1, 2, 5, 6], [1, 2, 5, 7], [1, 2

In [29]:
# Combinações com os filtros das restrições aplicados.

combinacoes_filtradas = combinaveis.enforce_constraints(combinacoes)
len(combinacoes_filtradas)

96

In [30]:
# Aplicándose o método detail_all_packages são obtidos os pacotes de investimento
# com os respetivos custos totais e retornos totais esperados.

pacotes_detalhados = combinaveis.detail_all_packages(combinacoes_filtradas)
len(pacotes_detalhados)

96

In [31]:
# Com base de entrada nos três elementos fundamentais da classe (gasto máximo incluso) 
# e o conjunto particular de pacotes detalhados,
# é aplicada a remoção de elementos fora do orçamento.
pacotes_no_orcamento = combinaveis.remove_off_budget_packages(pacotes_detalhados)
len(pacotes_no_orcamento)

61

In [32]:
# Recebendo os pacotes dentro do orçamento, o método optimize acha o pacote ótimo.
otimo = combinaveis.optimize(pacotes_no_orcamento)
print(otimo)

[[2, 4, 6, 7], 950000, 990000]


In [33]:
# Utilizando o método get_optimal é obtido o pacote de investimentos ótimo,
# só recebendo as informações de entrada.

InvestmentsSet(proposals,max_expenses,constraints_group).get_optimal()

[[2, 4, 6, 7], 950000, 990000]

## Testes

In [34]:
fake_set1 = [
    [1, 'investimento1', 90, 60],
    [2, 'investimento2', 50, 20],
    [3, 'investimento3', 30, 20],
    [4, 'investimento1', 40, 30]
    ]

fake_set2 = [
    [1, 'investimento1', 100, 60],
    [2, 'investimento2', 50, 20],
    [3, 'investimento3', 80, 20]
    ]

fake_budget = 150

fake_constraints = [constraint1, constraint2]

fake_combinations = [[1,5],[2,3],[2,4]]

fake_packages = [[1,2],[2,3,4]]

fake_detailed_package = [[[1,2,3],170,100],[[2,3,4],120,70],[[1,4],130,90]]

fake_on_budget_package = [[[2,3,4],120,70],[[1,4],130,90]]

In [42]:
class TestInvestmentsSet(unittest.TestCase):
  def setUp(self):
    self.Investments1 = InvestmentsSet(fake_set1, fake_budget, fake_constraints)
    self.Investments2 = InvestmentsSet(fake_set2, fake_budget, fake_constraints)

  def test_get_elements_combinations(self):
    self.assertEqual(self.Investments1.get_elements_combinations, [1,2,3,4])
  
  def test_min_number_elements(self):
    self.assertEqual(self.Investments1.min_number_elements, 2)
  
  def test_max_number_elements(self):
    self.assertEqual(self.Investments1.max_number_elements, 3)
  
  def test_combine(self):
    self.assertEqual(self.Investments2.combine, [[1],[2],[3],[1,2],[1,3],[2,3]])
  
  def test_enforce_constraints(self):
    self.assertEqual(self.Investments1.enforce_constraints(fake_combinations), [[2,4]])
  
  def test_detail_all_packages(self):
    self.assertEqual(self.Investments1.detail_all_packages(fake_packages), [[[1,2],140,80],[[2,3,4],120,70]])
  
  def test_remove_off_budget_package(self):
    self.assertEqual(self.Investments1.remove_off_budget_packages(fake_detailed_package), [[[2,3,4],120,70],[[1,4],130,90]])
  
  def test_optimize(self):
    self.assertEqual(self.Investments1.optimize(fake_on_budget_package), [[1,4],130,90])

  def test_get_optimal(self):
    self.assertEqual(self.Investments1.get_optimal(), [[1,4],130,90])

 
if __name__ == '__name__':
  unittest.main()

In [43]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_combine (__main__.TestInvestmentsSet) ... ok
test_detail_all_packages (__main__.TestInvestmentsSet) ... ok
test_enforce_constraints (__main__.TestInvestmentsSet) ... ok
test_get_elements_combinations (__main__.TestInvestmentsSet) ... ok
test_get_optimal (__main__.TestInvestmentsSet) ... ok
test_max_number_elements (__main__.TestInvestmentsSet) ... ok
test_min_number_elements (__main__.TestInvestmentsSet) ... ok
test_optimize (__main__.TestInvestmentsSet) ... ok
test_remove_off_budget_package (__main__.TestInvestmentsSet) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.021s

OK
